In [20]:
from IPython.display import Markdown, display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3 as sq

In [21]:
con = sq.connect("telecom.db")
cur = con.cursor()

1. Retrieve table names from the database (telecom.db).

In [22]:
tables = cur.execute("""select name from sqlite_master where type='table'""")
tables.fetchall()

[('train',),
 ('event_type',),
 ('severity_type',),
 ('resource_type',),
 ('log_feature',)]

2. Join all tables in the database on the primary key.

In [23]:
pd.read_sql("""select lf.id, lf.log_feature, lf.volume, et.event_type, rt.resource_type, st.severity_type, t.location, t.fault_severity
                from log_feature as lf 
                left join event_type as et 
                    on et.`index` = lf.`index`
                left join resource_type as rt
                    on et.`index` = rt.`index`
                left join severity_type as st
                    on et.`index` = st.`index`
                left join train as t
                    on et.id = t.id""", con)

,id,log_feature,volume,event_type,resource_type,severity_type,location,fault_severity
0,6597,feature 68,6,event_type 11,resource_type 8,severity_type 2,None,NaN
1,8011,feature 68,7,event_type 15,resource_type 8,severity_type 2,location 1,0.0
2,2597,feature 68,1,event_type 15,resource_type 8,severity_type 2,None,NaN
3,5022,feature 172,2,event_type 15,resource_type 8,severity_type 1,None,NaN
4,5022,feature 56,1,event_type 11,resource_type 8,severity_type 1,None,NaN
...,...,...,...,...,...,...,...,...
58666,8720,feature 209,1,None,None,None,None,NaN
58667,6488,feature 54,3,None,None,None,None,NaN
58668,878,feature 62,1,None,None,None,None,NaN
58669,4464,feature 209,1,None,None,None,None,NaN


3. Find unique occurrences of event_type and severity using an SQL query.

In [24]:
pd.read_sql("""select distinct event_type, severity_type
                from log_feature as lf 
                left join event_type as et 
                    on et.`index` = lf.`index`
                left join resource_type as rt
                    on et.`index` = rt.`index`
                left join severity_type as st
                    on et.`index` = st.`index`
                left join train as t
                    on et.id = t.id""", con)

,event_type,severity_type
0,event_type 11,severity_type 2
1,event_type 15,severity_type 2
2,event_type 15,severity_type 1
3,event_type 11,severity_type 1
4,event_type 20,severity_type 2
...,...,...
147,event_type 43,None
148,event_type 52,None
149,event_type 51,None
150,event_type 48,None


4. Find how many occurrences there are of each fault_severity using an SQL query.

In [26]:
pd.read_sql("""select fault_severity, count(*) as counts
                from log_feature as lf 
                left join event_type as et 
                    on et.`index` = lf.`index`
                left join resource_type as rt
                    on et.`index` = rt.`index`
                left join severity_type as st
                    on et.`index` = st.`index`
                left join train as t
                    on et.id = t.id
                group by fault_severity""", con)

,fault_severity,counts
0,NaN,46203
1,0.0,8065
2,1.0,3140
3,2.0,1263


In [32]:
pd.read_sql("""select fault_severity, count(*) as counts
                from train
                group by fault_severity""", con)

,fault_severity,counts
0,0,4784
1,1,1871
2,2,726


One of these is probably right